In [1]:
import altair as alt
import pandas as pd

In [60]:
def visualize_poll(poll_obj, show_user=False, show_duration=False, show_date=False):
    '''
    Returns a simple bar chart of poll responses
    Option to include additional information in the text box

    Parameters
    ----------
    poll_obj : dictionary
        the output of get_poll_by_id() function
    show_user : bool
        option to display user handle in the textbox
        default = False
    show_duration : bool
        option to display poll duration
        default = False
    show_date : bool
        option to display date
        default = False

    Returns
    --------
        an altair bar chart for the poll responses
        includes a textbox with additional information if at least one of
        - show_user
        - show_duration
        - show_date
        was set to True

    Examples
    --------
    >>> visualize_poll('4235234', show_duration=True)

    '''
    # Check for valid inputs
    if not isinstance(poll_obj, dict):
        raise Exception("The type of the argument 'poll_obj' mush be a dictionary")
    
    # convert dictionary to pd.DataFrame
    df = pd.DataFrame(poll_obj["poll options"])
    
    # extract user id and print
    if show_user == True:
        print(f"The user of the poll: {poll_obj['user']}")
    
    # extract poll date and print
    if show_date == True: 
        print(f"The end date and time of the poll: {pd.Timestamp(poll_obj['date']).strftime('%Y-%m-%d %H:%M:%S')}") # len()=1
    
    # extract duration and print
    if show_duration == True:
        print(f"The duration of the poll in hours: {poll_obj['duration'] / 60:.1f}h")
    
    plot = alt.Chart(
        df, 
        title=alt.TitleParams(
            text=poll_obj['text'],
            anchor="start"
        )).mark_bar().encode(
        alt.Y('label', title='', sort='x'),
        alt.X('votes', title='Votes'),
        alt.Color('label',title='Options'),
        alt.Tooltip('votes')
    ).configure_axis(
        labelFontSize=15,
        titleFontSize=15,
    ).configure_title(fontSize=20
                     ).properties(
        height=200, width=400
    )
    
    return plot

In [ ]:
poll_date = ""
len(poll_date)

In [ ]:
sample_poll_obj = {
    "poll question": "What's your favorate pet?",
    "poll options": [
        {"label": "Cat",
        "votes": 156},
        {"label": "Dog",
        "votes": 256},
        {"label": "Guinea pig",
        "votes": 56}
    ],
    "duration": 1440,
    "date": "2019-11-28T20:26:41.000Z"
}

In [ ]:
df = pd.DataFrame(sample_poll_obj["poll options"])
df

In [ ]:
visualize_poll(sample_poll_obj, show_duration=True, show_date=True, show_total_responses=True)

In [ ]:
import os
import re
import numpy as np

def get_polls_from_user(username, num=5):
    '''
    Get list of poll ids for a given Twitter user
    Parameters
    ----------
    username : str
        username of the twitter user
    num : int
        number of polls to return, in desc chronological order
        default = 5
    Returns
    --------
        array of twitter ids
    Examples
    --------
    >>> get_polls_from_user('ChipotleTweets', 3)
    
    '''
    
    # check argument validity
    if not(isinstance(username, str)):
        raise TypeError('Invalid argument type: username must be a string.')
    elif not(isinstance(num, int)):
        raise TypeError('Invalid argument: input n_tweets must be >= 0.')

    # Twitter API credentials
    try:
        consumer_key = '4tuMyUqRb6oTR99QSqp32KR3V'
        consumer_secret = 'O4GgyHAMsIE70YxbUBQ0pizV4gnL8JVek8Jy2LeYK4h908bUX2'
        access_key = '1483563822497492993-8NIPFIRy0AlHdn35iBrCUcg5EW56nU'
        access_secret = '9o3Tgy7sUt2mEa37wTKnUSj8xBC7TRUYtX8RCq4R9FFOX'
    except KeyError:
        raise Exception('Need authentication tokens! Please make sure you have those as environment variables.')

    auth = tweepy.OAuthHandler(consumer_key, consumer_secret)
    auth.set_access_token(access_key, access_secret)
    api = tweepy.API(auth)

    # get first batch of tweets
    tweets = []
    latest = api.user_timeline(screen_name=username,
                                exclude_replies=True)
    tweets.extend(latest)
        
    # request recursively to get all tweets
    oldest = latest[-1].id
    while(len(latest) > 0 and len(tweets) < num):
        latest = api.user_timeline(screen_name=username,
                                   exclude_replies=True,
                                   count=200, 
                                   max_id=oldest)
        tweets.extend(latest)
        
        #for tweet in tweets:
        #    if "polls" in tweet.entities:
        #        if tweet.entities["polls"] != "[]":
        #            tweets_polls.extend(latest)
        #    else:
        #        pass
        
        oldest = latest[-1].id # TODO: error out sometimes
        
    # format output dataframe   
    output = pd.DataFrame([[tweet.created_at, tweet.text] for tweet in tweets],
                          columns=['time', 'tweet'])
    output = output[:num]

    return output

get_polls_from_user('ElonMusk')

In [48]:
def get_poll_by_id(tweet_id):
    '''
    Extracts poll data from Twitter given the poll ID
    Parameters
    ----------
    tweet_id : str
        id of the tweet containing twitter poll
    
    Returns
    --------
    a dictionary with the following keys:
        poll question,
        poll responses,
        total votes,
        duration,
        date    
    
    Examples
    --------
    >>> get_poll_by_id('4235234')
    '''

    if not(isinstance(tweet_id, int)):
        raise TypeError('Invalid argument type: poll id must be numeric string.')

   # Twitter API credentials
    # from dotenv import load_dotenv
    # load_dotenv()
    # bearer_token = os.environ.get("BEARER_TOKEN")
    bearer_token = 'AAAAAAAAAAAAAAAAAAAAAAyIYQEAAAAAjvBdCMMh1dT8clkpXhHxzld7Dhs%3DLPl5zMXXOZqznZGe9JP7zHj3Wzx0N4unogLcWl8wfIkwikjQKm'
    
    client = tweepy.Client(bearer_token=bearer_token)

    res_tweet = client.get_tweets(tweet_id, expansions=["attachments.poll_ids", "author_id"], poll_fields=["duration_minutes","end_datetime"])
    res = res_tweet.includes

    try:
        res['polls'][0]
    except KeyError:
         raise TypeError('Provided tweet does not contain a poll!')

    poll = res['polls'][0]
    duration = poll['duration_minutes']
    date = poll['end_datetime']
    options = poll['options']
    text = res_tweet.data[0]['text']
    user = res['users'][0].username

    total = 0
    for opt in options:
        total = total + opt['votes']

    rtn = {
        "text" : text,
        "duration" : duration,
        "date" : date,
        "poll options" : options,
        "user" : user,
        "total" : total
    }

    return rtn


In [51]:
sample = get_poll_by_id(1484375486473986049)
sample

{'text': 'Important research!!!',
 'duration': 1440,
 'date': '2022-01-22T04:01:08.000Z',
 'poll options': [{'position': 1, 'label': 'Cookies', 'votes': 29},
  {'position': 2, 'label': 'Cupcakes', 'votes': 5},
  {'position': 3, 'label': 'Donuts', 'votes': 24},
  {'position': 4, 'label': 'Ice Cream', 'votes': 25}],
 'user': 'GregShahade',
 'total': 83}

In [61]:
visualize_poll(sample, show_user=True, show_duration=True, show_date=True)

The user of the poll: GregShahade
The end date and time of the poll: 2022-01-22 04:01:08
The duration of the poll in hours: 24.0h


alt.Chart(...)

In [52]:
sample_poll_obj = {
    "text": "What's your favorate pet?",
    "duration": 1440,
    "date": "2019-11-28T20:26:41.000Z",
    "poll options": [
        {"label": "Cat",
        "votes": 156},
        {"label": "Dog",
        "votes": 256},
        {"label": "Guinea pig",
        "votes": 56}
    ],
    
    
}

In [57]:
sample = {'text': 'Important research!!!',
 'duration': 1440,
 'date': '2022-01-22T04:01:08.000Z',
 'poll options': [{'position': 1, 'label': 'Cookies', 'votes': 29},
  {'position': 2, 'label': 'Cupcakes', 'votes': 5},
  {'position': 3, 'label': 'Donuts', 'votes': 24},
  {'position': 4, 'label': 'Ice Cream', 'votes': 25}],
 'user': 'GregShahade',
 'total': 83}

In [58]:
sample

{'text': 'Important research!!!',
 'duration': 1440,
 'date': '2022-01-22T04:01:08.000Z',
 'poll options': [{'position': 1, 'label': 'Cookies', 'votes': 29},
  {'position': 2, 'label': 'Cupcakes', 'votes': 5},
  {'position': 3, 'label': 'Donuts', 'votes': 24},
  {'position': 4, 'label': 'Ice Cream', 'votes': 25}],
 'user': 'GregShahade',
 'total': 83}

In [59]:
pd.DataFrame(sample["poll options"])

,position,label,votes
0,1,Cookies,29
1,2,Cupcakes,5
2,3,Donuts,24
3,4,Ice Cream,25
